In [21]:
from transformers import AutoTokenizer, BertTokenizer, BertForQuestionAnswering, TrainingArguments, Trainer,TrainingArguments, AutoModelForSequenceClassification
from datasets import load_dataset
import torch


In [22]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')

# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
# outputs = model(**inputs, labels=labels)
# loss = outputs.loss
# logits = outputs.logits

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\kjp19/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at C:\U

In [23]:
dataset = load_dataset('json', data_files={'train': 'train.jsonl', 'test': 'test.jsonl'})


print(dataset)

Using custom data configuration default-930d0c42b71c815f
Reusing dataset json (C:\Users\kjp19\.cache\huggingface\datasets\json\default-930d0c42b71c815f\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['word', 'sentence1', 'sentence2', 'idx', 'label', 'start1', 'start2', 'end1', 'end2', 'version'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['word', 'sentence1', 'sentence2', 'idx', 'label', 'start1', 'start2', 'end1', 'end2', 'version'],
        num_rows: 638
    })
})


In [24]:
def preprocess_data(examples):
    # encode a batch of sentences
    encoding = tokenizer(examples["sentence1"], padding="max_length", truncation=True)
    # add labels as a list
    encoding["labels"] = examples["label"]
    
    return encoding


# tokenize sentences + add labels
encoded_dataset = dataset.map(preprocess_data)
# turn into PyTorch dataset
encoded_dataset.set_format("torch")


small_train_dataset = encoded_dataset["train"].shuffle(seed=42).select(range(500))
small_eval_dataset = encoded_dataset["test"].shuffle(seed=42).select(range(500))
print(encoded_dataset)

  0%|          | 0/5000 [00:00<?, ?ex/s]

  0%|          | 0/638 [00:00<?, ?ex/s]

DatasetDict({
    train: Dataset({
        features: ['word', 'sentence1', 'sentence2', 'idx', 'label', 'start1', 'start2', 'end1', 'end2', 'version', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['word', 'sentence1', 'sentence2', 'idx', 'label', 'start1', 'start2', 'end1', 'end2', 'version', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 638
    })
})


In [27]:
training_args = TrainingArguments("test_trainer", evaluation_strategy="epoch")

trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


TypeError: __init__() got an unexpected keyword argument 'num_labels'